In [1]:
import torch
import soundfile as sf
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.api import TTS

from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

c:\Users\jcan2\GitHub\ReelGenerator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Flash attention 2 is not installed


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
version = "2" # Used when specifying output path of audio files, increment as needed

In [13]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# generate speech by cloning a voice using default settings
tts.tts_to_file(text="It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
                file_path="./audios/ttsAPI" + version + ".wav",
                speaker_wav="./voices/longvoice1.wav",
                language="en")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
["It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent."]
 > Processing time: 23.671223640441895
 > Real-time factor: 3.8464691757438962


'./audios/ttsAPI2.wav'

In [14]:
MODEL_DIR = "./models/XTTS-v2"  # folder containing config.json, model.pth, speakers_xtts.pth, vocab.json, etc.

config = XttsConfig()
config.load_json(f"{MODEL_DIR}/config.json")

xttsModel = Xtts.init_from_config(config)
xttsModel.load_checkpoint(config, checkpoint_dir=MODEL_DIR, eval=True)

xttsModel = xttsModel.to(device)

outputs = xttsModel.synthesize(
    "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
    config,
    speaker_wav="./voices/longvoice1.wav",
    language="en",
)

sf.write("./audios/ttsSaved" + version + ".wav", outputs["wav"], config.audio.sample_rate)

In [ ]:
parlerModel = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")

prompt = """
I've made so many mistakes.
I'm completely lost in life.
Everything's falling apart.
Stop thinking negatively.
The past is in the past.
The future is in the future.
I just need to focus on today.
"""
description = "A burly man's voice is monotone yet warm, deep, inspiring, with a very close recording that almost has no background noise."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = parlerModel.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("./audios/parler_tts_out.wav", audio_arr, parlerModel.config.sampling_rate)